## JSON files

In [1]:
# We'll load a couple of libraries that we'll be using in this Notebook.
# 'requests' is an HTTP library - it allows us to get data from a URL address
import requests
import json
import pandas as pd

## Using the JSON library
The Python `json` library provides a method, `json.loads()`, for parsing a string containing JSON data, that has been read in from a file via a web address, and converting it to a `dict`.

In [2]:
# We start by getting the data from the web address - we can inspect its content directly.
# This is data from the Ordanance Survey datasets.
url = 'http://data.ordnancesurvey.co.uk/datasets/os-linked-data/apis/reconciliation?query=MK7&type=http%3A%2F%2Fdata.ordnancesurvey.co.uk%2Fontology%2Fpostcode%2FPostcodeSector&type_strict=any&limit=10'
resp = requests.get(url)
resp.content

b'<!doctype html><html itemscope itemtype="http://schema.org/" lang="en-GB" dir="ltr"><head><meta charset="UTF-8"/><meta name="viewport" content="width=device-width,initial-scale=1"/><meta name="application-name" content="Ordnance Survey"/><link rel="stylesheet" href="/static-assets/css/osweb-styles.9c33187ef9d851b1a152.css"/><title data-react-helmet="true">Linked data | Data Products | OS</title> <meta data-react-helmet="true" name="msapplication-TileColor" content="var(--osds-colour-primary-1)"/><meta data-react-helmet="true" name="msapplication-config" content="/static-assets/browserconfig.xml"/><meta data-react-helmet="true" name="theme-color" content="var(--osds-colour-primary-1)"/><meta data-react-helmet="true" name="description" content="The OS Linked Data service has now been withdrawn. OS recommends a migration to the various API services provided by the OS Data Hub."/><meta data-react-helmet="true" property="og:site_name" content="Ordnance Survey"/><meta data-react-helmet="tr

Sometimes we need to decode the data that is returned.

If the response is preceded by a `b'`, as above, then we need to convert from *bytes* to a string representation by *decoding* the response.

In [3]:
data = json.loads(resp.content.decode('UTF8'))
data

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

The json library is quite short; additional documentation can be found at http://docs.python-guide.org/en/latest/scenarios/json/

## *pandas* can handle JSON too

The *pandas* `io.json` library has the `read_json()` function - it too reads from a web address.

In [ ]:
# The pandas function seems to handle the decoding seemlessly.
# Can you see how the JSON data structure is mapped on to the shape of the resulting DataFrame?
pd.io.json.read_json(url)

We can then convert the resulting DataFrame (structured as a dict in each row, in case you hadn't spotted it) into a more regular DataFrame format.

In [ ]:
# Here's one way of converting that DataFrame, 
#     with a dict generated from the JSON in each row, to a DataFrame.
d = pd.io.json.read_json(url)
# Create a list to handle the DataFrame generated by each row.
dfl=[]
for row in d['result']:
    # Convert the dict in each row to a DataFrame.
    dfl.append(pd.DataFrame.from_dict(row))
# Concatenate all the DataFrames:
pd.concat(dfl)

Alternatively, we can use the json library `json_loads()` output with the *pandas* `json_normalise()` function.

In [ ]:
# There is a handy pandas io function that can help us normalise this data:
from pandas.io.json import json_normalize
json_normalize(data['result'])

In [ ]:
# Let's have a look at the JSON from a BBC programme episode page 
#   such as http://www.bbc.co.uk/programmes/p01ztvcp.
# NOTE- The ability to access this data directly may be removed at some future date. 
# The BBC is moving to the NITRO API http://developer.bbc.co.uk/content/nitro-quickstart
#    when that happens, access to the content will require registering as a developer.

url = 'http://www.bbc.co.uk/programmes/p01ztvcp.json'
resp = requests.get(url)
data = json.loads(resp.content.decode('utf8'))
data

In [ ]:
# As seen before, the read_json() function can read and parse this data directly.
# But, the above is a nested structure.
# Can you predict what sort of shape the resulting DataFrame is likely to take?
tmp = pd.io.json.read_json('http://www.bbc.co.uk/programmes/p01ztvcp.json')
tmp

Not every JSON structure will map easily to a tablular form.  The structure above has several levels of nesting, but there are a few items at the top level that you can pull out.

For deeply nested data structures that do not have a natural tabular representation, it probably makes sense to parse them using 'standard' Python methods - handling each row individually, or grouping commonly structured rows together. 

In [ ]:
# For example, we can select particular rows by index value:
rows = ['pid', 'title', 'short_synopsis', 'long_synopsis', 'first_broadcast_date']
pd.io.json.read_json('http://www.bbc.co.uk/programmes/p01ztvcp.json').ix[rows]

There is some degree of control over the way in which the *pandas* `read_json()` function can parse imported data.

See the [pandas.io.json.read_json](http://pandas.pydata.org/pandas-docs/dev/generated/pandas.io.json.read_json.html) documentation for more information (particularly the `orient` parameter). The documentation also describes parameters for enabling automatic data conversions, where possible.

### Writing DataFrames to JSON files

We can write a DataFrame to a JSON file using the `to_json()` method.

In [ ]:
tmp.to_json('data/tmp.json')
!head data/tmp.json

See the *pandas* documentation for more details: [pandas.DataFrame.to_json](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_json.html).

## Summary
In this Notebook you have seen how to:
1.  read JSON data using the `json.loads()` function
2.  parse JSON data using *pandas*
3.  use *pandas* to parse and manipulate JSON data
4.  write data in a DataFrame to a JSON file.

## What next?

If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, move on to look at `02.2.3 Data file formats - other`. 